In [1]:
import numpy as np

from tensorflow import keras

In [2]:
def load_images(directory, image_size, batch_size=16, validation_split=0.25, seed=2704):
    print(f'Cargando imágenes desde: {directory}')

    train_data = keras.utils.image_dataset_from_directory(
        directory,
        validation_split = validation_split,
        subset = "training",
        # categorical hace que la salida sea sparce (one hot encoding)
        label_mode='categorical',
        seed = seed,
        image_size = image_size,
        batch_size = batch_size,
        shuffle=True)
    
    valid_data = keras.utils.image_dataset_from_directory(
        directory,
        validation_split = validation_split,
        subset = "validation",
        # categorical hace que la salida sea sparce (one hot encoding)
        label_mode='categorical',
        seed = seed,
        image_size = image_size,
        batch_size = batch_size)
    
    print("\n¡Imágenes cargadas exitosamente!")
    print(f"Número de clases encontradas: {len(train_data.class_names)}")
    print(f"Nombres de las clases: {train_data.class_names}")
    
    normalization_model = keras.layers.Rescaling(1./255)
    print("Capa normalizadora creada con éxito") 

    augmentation_model = keras.models.Sequential()
    # aumentamos el número de imágenes haciendo imágenes en espejo vertical y horizontal
    augmentation_model.add(keras.layers.RandomFlip("horizontal_and_vertical")) 
    # aumentamos haciendo rotaciones de las imágenes.
    # factor hace que demos un giro aleatorio desde -0.25 a 0.25 vueltas.
    # fill_mode hace que rellenemos los pixeles vacíos con nearest (rellenamos con el pixel que tenemos mas cerca)
    augmentation_model.add(keras.layers.RandomRotation(factor=0.25, fill_mode='nearest'))
    augmentation_model.add(keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode='nearest'))
    print("Capas de aumento de imágenes creadas con éxito")

    return train_data, valid_data, normalization_model, augmentation_model

In [3]:
def image_augmentation(image, label, augmentation_model):
    image = augmentation_model(image)
    return image, label

In [4]:
def image_normalization(image, label, normalization_model):
    image = normalization_model(image)
    return image, label

In [5]:
def create_model(imput_shape):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=imput_shape))
    model.add(keras.layers.Conv2D(filters = 6, kernel_size = (5,5), activation = "relu", padding = "same"))
    model.add(keras.layers.AvgPool2D(pool_size=(2,2)))
    model.add(keras.layers.Conv2D(filters = 16, kernel_size = (5,5), activation = "relu", padding = "valid"))
    model.add(keras.layers.AvgPool2D(pool_size=(2,2)))
    model.add(keras.layers.Conv2D(filters = 16, kernel_size = (5,5), activation = "relu", padding = "valid"))
    model.add(keras.layers.AvgPool2D(pool_size=(2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=64, activation="relu"))
    model.add(keras.layers.Dense(units=64, activation="relu"))
    model.add(keras.layers.Dense(units=15, activation="softmax"))

    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

    model.summary()

    return model

In [6]:
def train_model(model, train_data, valid_data, batch_size=None, epochs=10, steps_per_epoch=None, validation_steps=None):
    model.fit(
        train_data,
        validation_data = valid_data,
        batch_size = batch_size,
        epochs = epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps = validation_steps,
        verbose = 2)
    
    return model

In [7]:
path = "archive"
image_size = (150, 150)
input_shape = (150, 150, 3)
batch_size=16
validation_split=0.25
seed=2704

# carga de datos
train_data, valid_data, normalization_model, augmentation_model = load_images(path, image_size=image_size, batch_size=batch_size, validation_split=validation_split, seed=seed)

# tratamiento a datos de entrenamiento
train_data = train_data.map(lambda image, label: image_augmentation(image, label, augmentation_model))
train_data = train_data.map(lambda image, label: image_normalization(image, label, normalization_model))

# tratamiento a datos de validación
valid_data = valid_data.map(lambda image, label: image_normalization(image, label, normalization_model))

model = create_model(input_shape)
model = train_model(model=model, train_data=train_data, valid_data=valid_data)

Cargando imágenes desde: archive
Found 70549 files belonging to 15 classes.
Using 52912 files for training.
Found 70549 files belonging to 15 classes.
Using 17637 files for validation.

¡Imágenes cargadas exitosamente!
Número de clases encontradas: 15
Nombres de las clases: ['Apple', 'Banana', 'Carambola', 'Guava', 'Kiwi', 'Mango', 'Orange', 'Peach', 'Pear', 'Persimmon', 'Pitaya', 'Plum', 'Pomegranate', 'Tomatoes', 'muskmelon']
Capa normalizadora creada con éxito
Capas de aumento de imágenes creadas con éxito


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 75, 75, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 71, 71, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 35, 35, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 31, 31, 16)     │         6,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 15, 15, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       230,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 244,887 (956.59 KB)

 Trainable params: 244,887 (956.59 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3307/3307 - 479s - 145ms/step - acc: 0.5248 - loss: 1.4170 - val_acc: 0.7066 - val_loss: 0.8530
Epoch 2/10
3307/3307 - 437s - 132ms/step - acc: 0.7317 - loss: 0.7840 - val_acc: 0.7836 - val_loss: 0.6393
Epoch 3/10
3307/3307 - 528s - 160ms/step - acc: 0.7840 - loss: 0.6309 - val_acc: 0.8169 - val_loss: 0.5251
Epoch 4/10
3307/3307 - 410s - 124ms/step - acc: 0.8040 - loss: 0.5679 - val_acc: 0.8251 - val_loss: 0.5034
Epoch 5/10
3307/3307 - 441s - 133ms/step - acc: 0.8195 - loss: 0.5162 - val_acc: 0.8391 - val_loss: 0.4510
Epoch 6/10
3307/3307 - 482s - 146ms/step - acc: 0.8284 - loss: 0.4895 - val_acc: 0.8589 - val_loss: 0.3979
Epoch 7/10
3307/3307 - 418s - 126ms/step - acc: 0.8360 - loss: 0.4609 - val_acc: 0.8696 - val_loss: 0.3736
Epoch 8/10
3307/3307 - 404s - 122ms/step - acc: 0.8463 - loss: 0.4351 - val_acc: 0.8706 - val_loss: 0.3649
Epoch 9/10
3307/3307 - 424s - 128ms/step - acc: 0.8565 - loss: 0.4070 - val_acc: 0.8736 - val_loss: 0.3448
Epoch 10/10
3307/3307 - 426s - 129ms/